## Advent of Code 2023, day3

The engineer explains that an engine part seems to be missing from the engine, but nobody can figure out which one. If you can add up all the part numbers in the engine schematic, it should be easy to work out which part is missing.

The engine schematic (your puzzle input) consists of a visual representation of the engine. There are lots of numbers and symbols you don't really understand, but apparently any number adjacent to a symbol, even diagonally, is a "part number" and should be included in your sum. (Periods (.) do not count as a symbol.)

Here is an example engine schematic:
```txt
467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..
```

In this schematic, two numbers are not part numbers because they are not adjacent to a symbol: 114 (top right) and 58 (middle right). Every other number is adjacent to a symbol and so is a part number; their sum is 4361.

Of course, the actual engine schematic is much larger. What is the sum of all of the part numbers in the engine schematic?

In [ ]:
from fastcore.utils import L,first
from aocd import get_data
inp = get_data(day=3, year=2023)
samp = '''467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..'''
samp

'467..114..\n...*......\n..35..633.\n......#...\n617*......\n.....+.58.\n..592.....\n......755.\n...$.*....\n.664.598..'

Idea: Check a "border" around each number if it contains a symbol.
```txt
467..114..
...*......
..35..633.
......#...
```
For `35` I'd need to scan this border:
```txt
467..114..
.####.....
.#35#.633.
.####.#...
```
- Use regex to match the number on the scanned line.
- Get the start coordinate and the length of the number.
- Check the "border"
- Add an empty line full of `.` at the top and at the bottom to simplify the processing.
- Add `.` at the beginning and the end of the line to simplify the processing.

When using a regex, can I get a starting position of the match?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Yes! When you use regex in Python, you can get the starting position of a match. Here are the main ways:

**Using `re.finditer()`** (most useful for your case):
```python
import re
for match in re.finditer(r'\d+', line):
    print(match.start(), match.group())  # start position and the matched text
```

**Using `match.start()` on a match object**:
```python
match = re.search(r'\d+', line)
if match:
    pos = match.start()  # starting position
    text = match.group()  # matched text
```

For your Advent of Code problem, `finditer()` is probably what you want since you need to find all numbers in each line and get their positions.

Does this help with your border-checking approach?

In [ ]:
xs = samp.splitlines()
x = xs[0]
x

'467..114..'

In [ ]:
import math
import re

for match in re.finditer(r'\d+', x):
    start = match.start()
    n = int(match.group())
    l = int(math.log10(n) + 1)

    print(n, start, l)  # start position and the matched text

467 0 3
114 5 3


In [ ]:

def finditer(*args, **kw): return list(re.finditer(*args, **kw))

ss = samp.replace('\n', '')
finditer('\d+', ss)

<>:4: SyntaxWarning: invalid escape sequence '\d'
<>:4: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipykernel_1749/3631499317.py:4: SyntaxWarning: invalid escape sequence '\d'
  finditer('\d+', ss)


[<re.Match object; span=(0, 3), match='467'>,
 <re.Match object; span=(5, 8), match='114'>,
 <re.Match object; span=(22, 24), match='35'>,
 <re.Match object; span=(26, 29), match='633'>,
 <re.Match object; span=(40, 43), match='617'>,
 <re.Match object; span=(57, 59), match='58'>,
 <re.Match object; span=(62, 65), match='592'>,
 <re.Match object; span=(76, 79), match='755'>,
 <re.Match object; span=(91, 94), match='664'>,
 <re.Match object; span=(95, 98), match='598'>]

In [ ]:
def convert_match(match):
    num = int(match.group())
    start = match.start()
    length = match.end() - match.start()
    return num, start, length

def get_matches(x):
    return L(re.finditer(r'\d+', x)).map(convert_match)

get_matches(x)

[(467, 0, 3), (114, 5, 3)]

In [ ]:
empty = '.'*(len(x)+2)
xs2 = L(xs).map(lambda s: "."+s+".")
xs2 = [empty] + xs2 + [empty]

for s in xs2: print(s)

............
.467..114...
....*.......
...35..633..
.......#....
.617*.......
......+.58..
...592......
.......755..
....$.*.....
..664.598...
............


In [ ]:
def add_border(lines):
    empty = '.'*(len(x)+2)
    return [empty] + L(lines).map(lambda s: "."+s+".") + [empty]

xs2 = add_border(xs)

for s in xs: print(s)
print()
for s in xs2: print(s)

467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..

............
.467..114...
....*.......
...35..633..
.......#....
.617*.......
......+.58..
...592......
.......755..
....$.*.....
..664.598...
............


In [ ]:
ys = L(range(len(xs2) - 2)).map(lambda i: xs2[i:i+3])
for x in ys: print(x)

['............', '.467..114...', '....*.......']
['.467..114...', '....*.......', '...35..633..']
['....*.......', '...35..633..', '.......#....']
['...35..633..', '.......#....', '.617*.......']
['.......#....', '.617*.......', '......+.58..']
['.617*.......', '......+.58..', '...592......']
['......+.58..', '...592......', '.......755..']
['...592......', '.......755..', '....$.*.....']
['.......755..', '....$.*.....', '..664.598...']
['....$.*.....', '..664.598...', '............']


In [ ]:

y = ys[0]
z = y[1]
z

'.467..114...'

In [ ]:
get_matches(z)

[(467, 1, 3), (114, 6, 3)]

In [ ]:
def get_part(s, start_index, length):
    return s[start_index-1:start_index+length+1]

print(get_part(z, 1, 3))
print(get_part(z, 6, 3))

.467.
.114.


In [ ]:
z[-1:0], z[10:11]

('', '.')

In [ ]:
def get_before(s, start_index):
    return s[start_index-1:start_index]

def get_after(s, start_index, length):
    return s[start_index+length:start_index+length+1]

print(get_before(z, 1), get_after(z, 1, 3))
print(get_before(z, 6), get_after(z, 6, 3))

. .
. .


In [ ]:
print(get_before('W467XY114Z.', 1))
print(get_after('W467XY114Z.', 1, 3))
print(get_before('W467XY114Z.', 6))
print(get_after('W467XY114Z.', 6, 3))

W
X
Y
Z


In [ ]:
def has_symbol(prev, curr, nxt, start_index, length):
    s = get_before(curr, start_index) + get_after(curr, start_index, length) + get_part(prev, start_index, length) + get_part(nxt, start_index, length)
    return any(c != '.' and not c.isdigit() for c in s)

def get_nums(prev, curr, nxt):
    return get_matches(curr).map(lambda t: (t[0], has_symbol(prev, curr, nxt, t[1], t[2])))

print(get_nums(y[0], y[1], y[2]))

[(467, True), (114, False)]


In [ ]:
ys.starmap(get_nums)

[[(467, True), (114, False)], [], [(35, True), (633, True)], [], [(617, True)], [(58, False)], [(592, True)], [(755, True)], [], [(664, True), (598, True)]]

In [ ]:
ys.starmap(get_nums).concat().filter(lambda t: t[1]).map(lambda t: t[0]).sum()

4361

In [ ]:
def solve(lines):
    xs = add_border(lines)
    ys = L(range(len(xs) - 2)).map(lambda i: xs[i:i+3])
    return ys.starmap(get_nums).concat().filter(lambda t: t[1]).map(lambda t: t[0]).sum()

solve(samp.splitlines())


4361

In [ ]:
solve(inp.splitlines())

550934

# Part 2

Idea:
- Convert the array with the added border into an array. Maybe use NumPy.
- The array will not contain digits - instead it will have the numeric value on each place of the number- for example, 467 will contain 

How can I create and initialize numpy 2d array?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

There are several ways to create and initialize a NumPy 2D array:

**Create with specific values:**
```python
import numpy as np
arr = np.array([[1, 2, 3], [4, 5, 6]])
```

**Create filled with zeros:**
```python
arr = np.zeros((rows, cols))  # e.g., np.zeros((10, 12))
```

**Create filled with a specific value:**
```python
arr = np.full((rows, cols), value)  # e.g., np.full((10, 12), -1)
```

**Create empty (uninitialized):**
```python
arr = np.empty((rows, cols))
```

For your Advent of Code problem, which approach are you thinking of using? Are you planning to fill it with the actual number values at each position where a number appears?

In [ ]:
import numpy as np
arr = np.zeros((len(xs2), len(xs2[0])))
arr

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [ ]:
xs2.map(get_matches)

[[], [(467, 1, 3), (114, 6, 3)], [], [(35, 3, 2), (633, 7, 3)], [], [(617, 1, 3)], [(58, 8, 2)], [(592, 3, 3)], [(755, 7, 3)], [], [(664, 2, 3), (598, 6, 3)], []]

In [ ]:
for ri, r in enumerate(xs2.map(get_matches)):
    for ci, c in enumerate(r):
        for i in range(c[2]):
            arr[ri, ci + c[1] + i] = c[0]

arr

array([[  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.],
       [  0., 467., 467., 467.,   0.,   0.,   0., 114., 114., 114.,   0.,
          0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.],
       [  0.,   0.,   0.,  35.,  35.,   0.,   0.,   0., 633., 633., 633.,
          0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.],
       [  0., 617., 617., 617.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  58.,  58.,   0.,
          0.],
       [  0.,   0.,   0., 592., 592., 592.,   0.,   0.,   0.,   0.,   0.,
          0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0., 755., 755., 755.,   0.,
          0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.],
       [  0.,   0., 664., 664., 664.,   0.,   0., 598., 598., 598.,   0.,
          0.],
       [  0.,   0.,  

Idea: Now find all asterisks and check the array if there are exactly 2 distinct numbers as neighbors.